Zadanie dotyczy wyszukiwania wzorców dwuwymiarowych.

1. Zaimplementuj algorytm wyszukiwania wzorca 2-wymiarowego 
2. Znajdź w załączonym pliku "haystack.txt" wszyskie sytuacje, gdy taka sama litera występuje na tej samej pozycji w dwóch kolejnych linijkach. Zwróć uwagę, na nierówną długość linii w pliku. 
3. Znajdź wszystkie wystąpienia "th" oraz "t h" w dwóch kolejnych liniach na tej samej pozycji. 
4. Wybierz przynajmniej 4 litery (małe). Znajdź wszystkie wystąpienia tej litery w załączonym pliku "haystack.png" 
5. Znajdź wszystkie wystąpienia słowa "p a t t e r n" w haystack.png. 
6. Porównaj czas budowania automatu i czas wyszukiwania dla różnych rozmiarów wzorca 
7. Podziel plik na 2, 4 i 8 fragmentów (w poziomie) i porównaj czas przeszukiwania 


## 1.


Funkcja zwracająca kolumny wzorca

In [ ]:
def get_distinct_columns(pattern):
    n = len(pattern[0]) # len of rows
    m = len(pattern) # len of columns
    
    distinct_columns = []
    indexes = []
    letters = set()
    
    for j in range(n):
        col = []
        for i in range(m):
            col += [pattern[i][j]]
            letters.add(pattern[i][j])
            
        if col in distinct_columns:
            idx = distinct_columns.index(col)
            indexes.append(idx)
        else:
            distinct_columns.append(col)
            indexes.append(len(distinct_columns) - 1)
            
    return distinct_columns, indexes, letters

In [ ]:
pat = [['b', 'c', 'b'], ['a', 'c', 'a'], ['a', 'b', 'a']]
# pat = [['a','a'], 
        #    ['b','a']]

cols,idx,letters = get_distinct_columns(pat)
print(cols)
print(idx)
print(letters)

Pionowy automat

In [ ]:
def vertical_automaton(cols, letters):
    transition_table = [{}]
    n = len(cols[0]) # len of rows
    m = len(cols) # len of distinct_columns
    words = [[]]
    states = [0] * m
    
    for j in range(n):
        for i in range(m):
            if cols[i][j] in transition_table[states[i]]:
                states[i] = transition_table[states[i]][cols[i][j]]
            else:
                transition_table[states[i]][cols[i][j]] = len(transition_table)
                words.append(words[states[i]] + [cols[i][j]])
                states[i] = len(transition_table)
                transition_table.append({})
                
                
    for i, row in enumerate(transition_table):
        for letter in letters:
            if letter not in row:
                suffix = (words[i] + [letter])[1:]
                state = 0
                
                for s in suffix:
                    state = transition_table[state].get(s, 0)
                    
                row[letter] = state
                               
    return transition_table, states


In [ ]:
vertical_automaton(cols, letters)


In [ ]:
def horizontal_automaton(pattern, letters):
    n = len(pattern)
    ret = []
    
    for state in range(n+ 1):
        ret.append({})
        for a in letters:
            next_state = min(n, state + 1)
            
            while True:
                if pattern[:next_state] == (pattern[:state] + [a])[state - next_state + 1 : state + 1]:
                    break
                next_state -= 1 

            ret[state][a] = next_state
            
    return ret


In [ ]:
horizontal_automaton(pat, letters)

Automat 2d

In [ ]:
def get_automatons(pattern):
    columns, indexes, letters = get_distinct_columns(pattern)
    
    vertical_trans_table, vertical_states = vertical_automaton(columns, letters)
    
    pattern = [vertical_states[indexes[i]] for i in range(len(indexes))]
    
    horizontal_trans_table = horizontal_automaton(pattern, vertical_states)
    horizontal_state = len(horizontal_trans_table) - 1
    
    return (vertical_trans_table, horizontal_trans_table, horizontal_state)

In [ ]:
def pattern_matching_2d(text, pattern):
    
    vertical_trans_table, horizontal_trans_table, horizontal_accepting_state = get_automatons(pattern)

    ret = []
    vertical_states = []
    
    for i, line in enumerate(text):
        
        # przycinanie linii
        if len(line) < len(vertical_states):
            vertical_states = vertical_states[:len(line)]
        elif len(vertical_states) < len(line):
            vertical_states = vertical_states + [0]*(len(line) - len(vertical_states))
            
           
        horizontal_state = 0
        # przejście po linii
        for j, letter in enumerate(line):

            # Zaktualizowanie stanu automatu pionowego dla danej kolumny
            if letter in vertical_trans_table[vertical_states[j]]:
                vertical_states[j] = vertical_trans_table[vertical_states[j]][letter]
            else:
                vertical_states[j] = 0

            # Sprawdzenie, czy nowy stan automatów pionowych znajduje się w automacie poziomym
            if vertical_states[j] in horizontal_trans_table[horizontal_state]:
                horizontal_state = horizontal_trans_table[horizontal_state][vertical_states[j]]
        
                # Sprawdzenie, czy aktualny stan jest równy wynikowemu
                if horizontal_state == horizontal_accepting_state:
                    ret.append((i - len(pattern) + 1, j - len(pattern[0]) + 1))
                    
            else:
                horizontal_state = 0

    return ret


In [ ]:
text = [['x','a','b','a','a'], 
        ['d','b','a','c','b']]
pattern = ['ab','ba']
pattern_matching_2d(text, pattern)


## 2.

In [ ]:
def print_res(res, pattern): 
    print("______________________________________________________________")
    print("pattern:\n", pattern)
    print("\nfound on indexes:\n", res if len(res) > 0 else "\nno patter found\n", "\n")


In [ ]:
with open('haystack.txt') as file:
    text = file.readlines()
for i in range(ord('a'), ord('z')+1):
    pattern = [chr(i), chr(i)]
    res = pattern_matching_2d(text, pattern)
    print_res(res, pattern)


## 3.

In [ ]:
pattern = ["th",
           "th"]
res = pattern_matching_2d(text, pattern)
print_res(res, pattern)

In [ ]:
pattern = ["t h",
           "t h"]
res = pattern_matching_2d(text, pattern)
print_res(res, pattern)

## 4.

In [ ]:
from PIL import Image
def convert(filename):
    image = Image.open(filename)
    pixels = list(image.getdata())
    width, height = image.size
    text = []
    i = width
    for pixel in pixels:
        if i == width:
            i = 0
            text.append([])
        text[-1].append(pixel[0])
        i += 1
    return text


In [ ]:
text = convert('haystack.png')
t = convert('t.png')
h = convert('h.png')
e = convert('e.png')

t

In [ ]:
res = pattern_matching_2d(text, t)
print(f"Amount of letters t found is: {len(res)}")

h

In [ ]:
res = pattern_matching_2d(text, h)
print(f"Amount of letters h found is: {len(res)}")

e

In [ ]:
res = pattern_matching_2d(text, e)
print(f"Amount of letters e found is: {len(res)}")

## 5.

Pattern search

In [ ]:
pattern = convert('pattern.png')


In [ ]:
res = pattern_matching_2d(text, pattern)

print(f"Amount of patterns found is: {len(res)}")